In [3]:

import numpy as np
import math
import matplotlib
import matplotlib.pyplot as plt
from glob import glob
import re

def vector_dist(v1, v2): # euclidean distance between 2 points
    return math.sqrt(np.sum([(v1[i] - v2[i])**2 for i in range(len(v1))]))

true_diag = [20, 59]
true_antidiag = [36, 35]
mlen = 3 # number states


In [16]:
def parser_arr(filename):
    # returns array of all traansitions and list of all class and transition thresholds
    transitions = []
    thresholds = []
    vidarrs = []
    with open(filename) as textfile:
        line = textfile.readline()
        #print('opened', line)
        while line:
            if 'Processing' in line:
                vidnum = line.split('pp_')[-1]
                vidnum = vidnum.split('.avi')[0]
                vidnum = vidnum.split('_')[0]
                vidnum = int(vidnum)
                #print(vidnum)
                line = textfile.readline()
            if 'T threshold'in line:
                thresh_set = []
                for word in line.split():
                    try:
                        thresh_set.append(float(word))
                    except ValueError:
                        pass
                    
                line = textfile.readline()
                #print('after t', line)
                if 'C threshold' in line:
                    for word in line.split():
                        try:
                            thresh_set.append(float(word))
                        except ValueError:
                            pass
                        
                
                    if len(thresholds) ==0 or thresholds[-1] != thresh_set:
                        #print(thresh_set)
                        thresholds.append(thresh_set)
                        vidnums=[]
                        vidarrs.append([[], vidnums])
                        vidsarr = np.zeros((mlen,mlen))
                        transitions.append(vidsarr)
                    else:
                        pass    
                    
                    line = textfile.readline()
                    #print('array lives here', line)
                    if '[[' in line:
                        arr = np.zeros((mlen,mlen))
                        #print(arr)
                        linenums = [int(s) for s in  re.findall(r'\d+',line)]
                        #print(linenums)
                        arr[0,:] = linenums

                        line = textfile.readline()
                        #print('array lives here', line)   

                        linenums = [int(s) for s in  re.findall(r'\d+',line)]
                        #print(linenums)
                        arr[1,:] = linenums

                        line = textfile.readline()
                        #print('array lives here', line)   

                        linenums = [int(s) for s in  re.findall(r'\d+',line)]
                        #print(linenums)
                        arr[2,:] = linenums

                        transitions[-1] += arr
                        vidarrs[-1][0].append(arr)
                        vidarrs[-1][1].append(vidnum)
                        #print(arr,'\n', transitions[-1])
                    else:
                        print("missing array")
                    #print(thresholds[-1])
                else:
                    print("missing C thresh")
                
                
                    
            line = textfile.readline()
            #print(line, 'moved on')
                    
                
            
    textfile.close()
    #print(transitions, thresholds)
    return([transitions, np.array(thresholds), vidarrs]) 


In [15]:
cols = [ 'mediumslateblue','fuchsia','darkviolet' ]
print(parser_arr(glob('./output 052*.txt')[0]))

1
10
11
12
13
14
15
2
3
4
6
7
8
9
1
10
11
12
13
14
15
2
3
4
6
7
8
9
1
10
11
12
13
14
[[array([[100.,   1.,  51.],
       [  2.,   1.,   0.],
       [ 59.,   0., 137.]]), array([[ 92.,   1.,  29.],
       [  2.,   1.,   0.],
       [ 37.,   0., 151.]]), array([[13.,  1., 46.],
       [ 2.,  0.,  0.],
       [50.,  0., 26.]])], array([[30.        , 22.        ],
       [30.        , 35.        ],
       [34.57142857, 22.        ]]), [[[array([[ 8.,  1., 38.],
       [ 2.,  1.,  0.],
       [38.,  0.,  0.]]), array([[3., 0., 1.],
       [0., 0., 0.],
       [2., 0., 4.]]), array([[ 0.,  0.,  0.],
       [ 0.,  0.,  0.],
       [ 1.,  0., 12.]]), array([[ 0.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 2.,  0., 12.]]), array([[4., 0., 6.],
       [0., 0., 0.],
       [7., 0., 2.]]), array([[1., 0., 0.],
       [0., 0., 0.],
       [1., 0., 3.]]), array([[0., 0., 0.],
       [0., 0., 0.],
       [1., 0., 7.]]), array([[56.,  0.,  1.],
       [ 0.,  0.,  0.],
       [ 2.,  0.,  6.]]), array(

In [21]:
#broad observations array
trans_runs= parser_arr(glob('./output 052*.txt')[0])
def ratio(row):
    sums = np.sum(row)
    if sums > 0:
        return row/sums
    else:
        return row
index = list(range(len(trans_runs[0])))

criteria = []

for i in index:
    x1 = []
    x2 = []
    x3 = []
    c_threshes = []
    t_threshes = []
    trans_matrices = []
    N_transitions = trans_runs[0][i]
    trans_threshes = trans_runs[1][i]
    vidarrs = trans_runs[2][i][0]
    vidnums = trans_runs[2][i][1]
    print(vidnums)
    inds=np.argsort(vidnums)
    print(inds)
    vidarrs = vidarrs[np.array(inds)]
    #print(vidarrs)
    vidarrs = np.cumsum(vidarrs, axis=0)
    t_threshes.append(trans_threshes[0])
    c_threshes.append(trans_threshes[1])
    criteria.append(str(int(trans_threshes[0]))+','+str(int(trans_threshes[1])))
    vidnums = list(range(len(vidarrs)))
    for vidnum in vidnums: 
        T_obb = vidarrs[vidnum]
        trans_matrices.append(T_obb)
        T_obb_norm = np.vstack([ratio(T_obb[i]) for i in range(len(T_obb))])
        #print(T_obb, '\n', np.around(T_obb_norm, 2), np.sum(T_obb_norm, axis=1))
        eigvals, eigvecs = np.linalg.eig(T_obb_norm.T)
        eig_one_index = [x for x in range(len(eigvals)) if eigvals[x]==max(eigvals)][0]
        eigvec_one = eigvecs[:, eig_one_index]
        evec_one = eigvec_one*100 / np.sum(eigvec_one)
        print(evec_one, np.sum(T_obb_norm, axis=0)*100/np.sum(T_obb_norm))
        x1.append(evec_one[0])
        x2.append(evec_one[1])
        x3.append(evec_one[2])
    fig,ax = plt.subplots()
    print(len(t_threshes), len(x1))        
    plt.plot(vidnums, x1,c = cols[0], ls = '-')
    plt.plot(vidnums,x2, c = cols[1], ls = '-')
    plt.plot(vidnums,x3, c = cols[2], ls = '-')

    plt.axhline(y=min(x1), c = cols[0], ls = ':')
    plt.axhline(y=min(x2), c = cols[1], ls = ':')
    plt.axhline(y=min(x3), c = cols[2], ls = ':')
    plt.axhline(y=max(x1), c = cols[0], ls = ':')
    plt.axhline(y=max(x2), c = cols[1], ls = ':')
    plt.axhline(y=max(x3), c = cols[2], ls = ':')
    plt.xlabel(criteria[i]+': thresholds', fontsize=18)
    plt.ylabel('observed probability of transition', fontsize=16)

[1, 10, 11, 12, 13, 14, 15, 2, 3, 4, 6, 7, 8, 9]
[ 0  7  8  9 10 11 12 13  1  2  3  4  5  6]


TypeError: only integer scalar arrays can be converted to a scalar index

In [ ]:
for vid vidlist
props = propa [all lsit befor evid], propb [all list before vid], propc [all list before vid]